# Training Models on Entire Merged Data

In [1]:
import pandas as pd

import numpy as np

import os

from sklearn.model_selection import train_test_split # Import train_test_split function

from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor # Import Random Forest Model

from sklearn.preprocessing import LabelEncoder


from sklearn.preprocessing import OrdinalEncoder 

import pickle

from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

from sklearn import svm

In [2]:
%%time
df_all = pd.read_csv('merged.csv')
os.system("printf '\a'")

/opt/anaconda3/lib/python3.8/site-packages/decorator.py:232: DtypeWarning: Columns (0,1,2,4,7,10,11,12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  return caller(func, *(extras + args), **kw)


CPU times: user 1min 5s, sys: 58.6 s, total: 2min 4s
Wall time: 2min 37s


0

In [3]:
df_all.head(2)

,percent_pload_pass,percent_capacity,accel_time,route,aircraft_group,fl_date,mkt_carrier,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,distance_y,arr_delay,crs_route,time_diff_crs_elapsed_route_time,speed
0,0.676118,0.805369,27,DSM-MSP,6,2018-01-01,DL,3405,13:20:00,14:42:00,82,232,92,82,0,5.52381
1,0.676118,0.805369,27,DSM-MSP,6,2018-01-01,DL,3756,06:45:00,08:09:00,84,232,244,84,0,5.39535


### Bootstrap Sampling

In [4]:
%%time
df_btstr = df_all.sample(frac = 0.005, random_state = 1)
os.system("printf '\a'")

CPU times: user 4.44 s, sys: 12.7 s, total: 17.2 s
Wall time: 23.5 s


0

In [5]:
%%time
df_btstr = df_btstr[~pd.to_numeric(df_btstr.percent_pload_pass,errors='coerce').isnull()]
os.system("printf '\a'")

CPU times: user 1.7 s, sys: 11.2 s, total: 12.9 s
Wall time: 21.4 s


0

In [6]:
types = {'percent_pload_pass':'float','percent_capacity':'float','accel_time':'float','route':'string',
         'aircraft_group':'float','fl_date':'string','mkt_carrier':'string','mkt_carrier_fl_num':'float',
         'crs_dep_time':'string','crs_arr_time':'string','crs_elapsed_time':'float','distance_y':'float',
         'arr_delay':'float','crs_route':'float','time_diff_crs_elapsed_route_time':'float','speed':'float'}

In [7]:
%%time
df_btstr = df_btstr.astype(types)
os.system("printf '\a'")

CPU times: user 1.9 s, sys: 5.83 s, total: 7.73 s
Wall time: 12.9 s


0

In [8]:
df_btstr.head(3)

,percent_pload_pass,percent_capacity,accel_time,route,aircraft_group,fl_date,mkt_carrier,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,distance_y,arr_delay,crs_route,time_diff_crs_elapsed_route_time,speed
861864,0.299858,0.412709,560.0,EWR-BDL,6.0,2018-01-03,UA,4958.0,08:30:00,09:40:00,70.0,116.0,1.0,70.0,0.0,4.142857
15766982,0.607021,0.812857,75.0,TPA-BNA,6.0,2018-12-14,WN,2161.0,06:55:00,07:45:00,110.0,612.0,-11.0,50.0,-60.0,7.463415
30987332,0.593843,0.825990,1328.0,BUR-SEA,6.0,2019-11-19,AS,1021.0,08:00:00,10:47:00,167.0,937.0,-14.0,167.0,0.0,7.045113


In [9]:
def format_time_string(x):
    if x[0] == '0':
        x = x[1:5]
        x = x.replace(':','')
        
    elif x[0] in '123456789':
        x = x[0:5]
        x = x.replace(':','')
    return x

In [10]:
time_fet = ['crs_dep_time','crs_arr_time']
df_btstr[time_fet[0]] = df_btstr[time_fet[0]].apply(format_time_string)
df_btstr[time_fet[1]] = df_btstr[time_fet[1]].apply(format_time_string)
df_btstr.head(3)

,percent_pload_pass,percent_capacity,accel_time,route,aircraft_group,fl_date,mkt_carrier,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,distance_y,arr_delay,crs_route,time_diff_crs_elapsed_route_time,speed
861864,0.299858,0.412709,560.0,EWR-BDL,6.0,2018-01-03,UA,4958.0,830,940,70.0,116.0,1.0,70.0,0.0,4.142857
15766982,0.607021,0.812857,75.0,TPA-BNA,6.0,2018-12-14,WN,2161.0,655,745,110.0,612.0,-11.0,50.0,-60.0,7.463415
30987332,0.593843,0.825990,1328.0,BUR-SEA,6.0,2019-11-19,AS,1021.0,800,1047,167.0,937.0,-14.0,167.0,0.0,7.045113


In [11]:
mean_p_Cap = df_btstr.groupby('mkt_carrier', as_index=False)['percent_capacity'].median()
mean_p_Cap

,mkt_carrier,percent_capacity
0,AA,0.847087
1,AS,0.820517
2,B6,0.829091
3,DL,0.864436
4,F9,0.883230
5,G4,0.865385
6,HA,0.764133
7,NK,0.863199
8,UA,0.853487
9,VX,0.835534


### Test data filtering

In [12]:
%%time
df = pd.read_csv('flights_test2020.csv')
df = df.reset_index()
df = df.drop(columns=['index','Unnamed: 0'])
# a list of features to drop
drop_list = ['branded_code_share','mkt_carrier','op_unique_carrier','tail_num',
             'origin_city_name','dest_city_name','op_carrier_fl_num','dup','flights',
             'origin_airport_id','dest_airport_id']
        
df = df.drop(columns=drop_list)
        
#combine origin & dest as route
df['origin'] = df['origin']+'-'+df['dest']
df = df.drop(columns=['dest'])
df = df.rename(columns={'origin':'route'})

#convert fl_date into date format
#df['fl_date'] = pd.to_datetime(df['fl_date'], unit='ms').dt.date

# convert crs_dep_time, crs_arr_time into datetime type
time_features = ['crs_dep_time','crs_arr_time']
        
# convert into string
df[time_features] = df[time_features].astype(str)

# fill the string into 4 digits only
df[time_features] = df[time_features].apply(lambda x:x.str.zfill(4))


os.system("printf '\a'")

CPU times: user 4.02 s, sys: 711 ms, total: 4.73 s
Wall time: 5.3 s


0

In [13]:
df.shape

(660556, 8)

In [14]:
df= df.rename(columns={'mkt_unique_carrier':'mkt_carrier'})

df['percent_capacity'] = np.where(df.mkt_carrier ==mean_p_Cap.mkt_carrier[0],mean_p_Cap.percent_capacity[0],np.nan)
df['percent_capacity'] = np.where(df.mkt_carrier ==mean_p_Cap.mkt_carrier[1],mean_p_Cap.percent_capacity[1],np.nan)
df['percent_capacity'] = np.where(df.mkt_carrier ==mean_p_Cap.mkt_carrier[2],mean_p_Cap.percent_capacity[2],np.nan)
df['percent_capacity'] = np.where(df.mkt_carrier ==mean_p_Cap.mkt_carrier[3],mean_p_Cap.percent_capacity[3],np.nan)
df['percent_capacity'] = np.where(df.mkt_carrier ==mean_p_Cap.mkt_carrier[4],mean_p_Cap.percent_capacity[4],np.nan)
df['percent_capacity'] = np.where(df.mkt_carrier ==mean_p_Cap.mkt_carrier[5],mean_p_Cap.percent_capacity[5],np.nan)
df['percent_capacity'] = np.where(df.mkt_carrier ==mean_p_Cap.mkt_carrier[6],mean_p_Cap.percent_capacity[6],np.nan)
df['percent_capacity'] = np.where(df.mkt_carrier ==mean_p_Cap.mkt_carrier[7],mean_p_Cap.percent_capacity[7],np.nan)
df['percent_capacity'] = np.where(df.mkt_carrier ==mean_p_Cap.mkt_carrier[8],mean_p_Cap.percent_capacity[8],np.nan)
df['percent_capacity'] = np.where(df.mkt_carrier ==mean_p_Cap.mkt_carrier[9],mean_p_Cap.percent_capacity[9],np.nan)
df['percent_capacity'] = np.where(df.mkt_carrier ==mean_p_Cap.mkt_carrier[10],mean_p_Cap.percent_capacity[10],np.nan)

In [15]:
%%time

df = df[df.mkt_carrier.isin(np.unique(df_btstr.mkt_carrier))]
df = df[df.mkt_carrier_fl_num.isin(np.unique(df_btstr.mkt_carrier_fl_num))]
#df = df[df.crs_dep_time.isin(np.unique(df_btstr.crs_dep_time))]
#df = df[df.crs_arr_time.isin(np.unique(df_btstr.crs_arr_time))]
#df = df[df.crs_elapsed_time.isin(np.unique(df_btstr.crs_elapsed_time))]
#df = df[df.distance.isin(np.unique(df_btstr.distance_y))]
#df = df[df.percent_capacity.isin(np.unique(df_btstr.percent_capacity))]


os.system("printf '\a'")

CPU times: user 390 ms, sys: 29.7 ms, total: 419 ms
Wall time: 498 ms


0

In [16]:
df.shape

(657910, 9)

### Label encoding for string features

In [17]:
%%time
# creating LabelEncoder object
from sklearn.preprocessing import LabelEncoder
import numpy as np
enc = LabelEncoder()

# encoding
dfe = pd.DataFrame()    # created empty dataframe for saving encoded values
for column in df_btstr.columns:
    dfe[column] = enc.fit_transform(df_btstr[column])
    
os.system("printf '\a'")
dfe.head()

CPU times: user 581 ms, sys: 65 ms, total: 646 ms
Wall time: 753 ms


,percent_pload_pass,percent_capacity,accel_time,route,aircraft_group,fl_date,mkt_carrier,mkt_carrier_fl_num,crs_dep_time,crs_arr_time,crs_elapsed_time,distance_y,arr_delay,crs_route,time_diff_crs_elapsed_route_time,speed
0,2463,1433,554,1406,0,2,8,4931,1182,1360,40,27,69,40,8,740
1,45591,28997,69,4390,0,347,10,2159,1087,1245,80,415,57,20,6,15387
2,40178,32387,1322,572,0,687,1,1019,1152,112,137,677,54,137,8,11432
3,4695,17485,25,1462,0,69,8,1876,161,416,356,1342,26,176,3,18033
4,49781,45507,930,2783,0,249,3,119,399,657,131,744,85,191,10,8610


In [18]:
#df_all.apply(LabelEncoder().fit_transform)
#df_all['fl_date'] = enc.fit_transform(df_all[['fl_date']])
#df_all['fl_date'] = enc.inverse_transform(df_all[['fl_date']])
#df_all['route'] = enc.fit_transform(df_all[['route']])
#df_all['mkt_carrier'] = enc.fit_transform(df_all[['mkt_carrier']])
#df_all['crs_dep_time'] = enc.fit_transform(df_all[['crs_dep_time']])
#df_all['crs_arr_time'] = enc.fit_transform(df_all[['crs_arr_time']])

### Feature Selection 

In [19]:
%%time
#split dataset in features and target variable
#features = ['percent_pload_pass','percent_capacity','accel_time','route','aircraft_group','fl_date','mkt_carrier','mkt_carrier_fl_num',
#            'crs_dep_time','crs_arr_time','crs_elapsed_time','distance_y','crs_route','time_diff_crs_elapsed_route_time','speed']
#features = ['percent_capacity','route','fl_date','mkt_carrier','mkt_carrier_fl_num',
#            'crs_dep_time','crs_arr_time','crs_elapsed_time','distance_y','crs_route','time_diff_crs_elapsed_route_time']

#features = ['percent_capacity','route','aircraft_group','fl_date','mkt_carrier','mkt_carrier_fl_num',
#            'crs_dep_time','crs_arr_time','crs_elapsed_time','distance_y','crs_route','time_diff_crs_elapsed_route_time','speed']

#features = ['accel_time','route','aircraft_group','fl_date','mkt_carrier','mkt_carrier_fl_num',
#            'crs_dep_time','crs_arr_time','crs_elapsed_time','distance_y','crs_route','time_diff_crs_elapsed_route_time','speed']

#features = ['accel_time','route','aircraft_group','fl_date','mkt_carrier','mkt_carrier_fl_num',
#            'crs_dep_time','crs_elapsed_time','distance_y','crs_route','time_diff_crs_elapsed_route_time']


#features = ['percent_capacity','accel_time','route','fl_date','mkt_carrier','mkt_carrier_fl_num',
#            'crs_dep_time','crs_arr_time','crs_elapsed_time','distance_y','crs_route',
#            'time_diff_crs_elapsed_route_time']


features = ['fl_date','mkt_carrier','mkt_carrier_fl_num','route','crs_dep_time','crs_arr_time','crs_elapsed_time','distance_y','percent_capacity']
X = dfe[features] # features
y = dfe.arr_delay # target

os.system("printf '\a'")

CPU times: user 31.7 ms, sys: 42.2 ms, total: 73.9 ms
Wall time: 129 ms


0

### Split data into train and test sets

In [20]:
%%time
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test
os.system("printf '\a'")

CPU times: user 58.3 ms, sys: 15.4 ms, total: 73.7 ms
Wall time: 138 ms


0

### Train model with all features

In [39]:
%%time
#Create a Gaussian Classifier
clf=RandomForestRegressor(n_estimators=100)

os.system("printf '\a'")

CPU times: user 3.08 ms, sys: 130 ms, total: 134 ms
Wall time: 162 ms


0

In [40]:
%%time

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

os.system("printf '\a'")

CPU times: user 1min 54s, sys: 1.87 s, total: 1min 56s
Wall time: 2min 9s


0

In [41]:
clf.score(X_train,y_train)

0.8561187463252321

### Cross validation with test

In [42]:
%%time

y_pred = clf.predict(X_test)

os.system("printf '\a'")

CPU times: user 2.58 s, sys: 35.8 ms, total: 2.62 s
Wall time: 2.85 s


0

In [43]:
%%time

r2 = r2_score(y_test,y_pred)
mean_sq_er = mean_squared_error(y_test,y_pred)
mean_abs_er = mean_absolute_error(y_test,y_pred)

print('r2 score for cross validation is: {}'.format(r2))
print('mean_squared_error score for cross validation is: {}'.format(mean_sq_er))
print('mean_absolute_error score for cross validation is: {}'.format(mean_abs_er))
os.system("printf '\a'")

r2 score for cross validation is: -0.016498757183181167
mean_squared_error score for cross validation is: 1922.742684341544
mean_absolute_error score for cross validation is: 24.444399702165335
CPU times: user 6.07 ms, sys: 3.48 ms, total: 9.54 ms
Wall time: 12.5 ms


0

### Train Test Data

In [45]:
fl_dates = pd.DataFrame(df.fl_date)
routes = pd.DataFrame(df.route)

In [46]:
df.head()

,fl_date,mkt_carrier,mkt_carrier_fl_num,route,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,percent_capacity
0,2020-01-01,WN,5888.0,ONT-SFO,1810,1945,95.0,363.0,0.823983
1,2020-01-01,WN,6276.0,ONT-SFO,1150,1320,90.0,363.0,0.823983
2,2020-01-01,WN,4598.0,ONT-SJC,2020,2130,70.0,333.0,0.823983
3,2020-01-01,WN,4761.0,ONT-SJC,1340,1455,75.0,333.0,0.823983
4,2020-01-01,WN,5162.0,ONT-SJC,0915,1035,80.0,333.0,0.823983


In [47]:
types = {'route':'string','fl_date':'string','mkt_carrier':'string','mkt_carrier_fl_num':'float',
         'crs_dep_time':'string','crs_arr_time':'string','crs_elapsed_time':'float','distance':'float'}

In [48]:
%%time
df = df.astype(types)
os.system("printf '\a'")

CPU times: user 127 ms, sys: 41.4 ms, total: 169 ms
Wall time: 199 ms


0

In [49]:
%%time
# creating LabelEncoder object
from sklearn.preprocessing import LabelEncoder
import numpy as np
enc = LabelEncoder()

# encoding
dfe_test = pd.DataFrame()    # created empty dataframe for saving encoded values
for column in df.columns:
    dfe_test[column] = enc.fit_transform(df[column])
    
os.system("printf '\a'")

dfe_test.head()

CPU times: user 1.18 s, sys: 108 ms, total: 1.28 s
Wall time: 1.37 s


,fl_date,mkt_carrier,mkt_carrier_fl_num,route,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,percent_capacity
0,0,9,5855,3946,893,1073,70,251,0
1,0,9,6234,3946,513,688,65,251,0
2,0,9,4572,3947,1023,1178,45,223,0
3,0,9,4735,3947,623,783,50,223,0
4,0,9,5130,3947,358,523,55,223,0


In [50]:
y_test_pred = clf.predict(dfe_test)


In [51]:
dfe_test['arr_delay_pred'] = y_test_pred

In [52]:
submit_df = dfe_test[['fl_date','mkt_carrier','mkt_carrier_fl_num','route','arr_delay_pred']]

In [53]:
submit_df.to_csv('submission_still_encoded_4.csv')

### Save model with all features

In [ ]:
#%%time

#filename = 'model_all_feat.sav'
#pickle.dump(clf,open(filename,'wb'))


#os.system("printf '\a'")

### SVM

In [ ]:
#%%time
#clf_svm = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
#clf_svm.fit(X_train, y_train)

#Predict the response for test dataset
#y_pred = clf_svm.predict(X_test)

#os.system("printf '\a'")

In [ ]:
#clf_svm.score(X_train, y_train)

In [ ]:
#%%time

#r2 = r2_score(y_test,y_pred)
#mean_sq_er = mean_squared_error(y_test,y_pred)
#mean_abs_er = mean_absolute_error(y_test,y_pred)

#print('r2 score for cross validation is: {}'.format(r2))
#print('mean_squared_error score for cross validation is: {}'.format(mean_sq_er))
#print('mean_absolute_error score for cross validation is: {}'.format(mean_abs_er))
#os.system("printf '\a'")